# Test storage re-assignment

In [1]:
!lamin init --storage mytest2 --schema bionty

✅ saved: User(uid='a3keNAVX', handle='jkobject', updated_at=2023-11-21 21:35:51 UTC)
✅ saved: Storage(uid='d9usNZRL', root='/home/ml4ig1/Documents code/scPRINT/notebooks/assessments/mytest2', type='local', updated_at=2023-11-21 21:35:51 UTC, created_by_id=1)
💡 loaded instance: jkobject/mytest2
💡 did not register local instance on hub


In [1]:
import lamindb as ln
import lnschema_bionty as lb

lb.settings.organism = "human"

💡 lamindb instance: jkobject/mytest2


In [3]:
files_census = ln.Artifact.using("laminlabs/cellxgene-census")
file = files_census.filter(description__icontains="tabula sapiens - lung").one()
file

File(uid='YdQ7CKrqy1YtXhs8Me58', key='cell-census/2023-07-25/h5ads/0d2ee4ac-05ee-40b2-afb6-ebb584caa867.h5ad', suffix='.h5ad', accessor='AnnData', description='Tabula Sapiens - Lung|0d2ee4ac-05ee-40b2-afb6-ebb584caa867', size=956851025, hash='hGVVgGbXP1IGDTI2PLg01A-115', hash_type='md5-n', visibility=0, key_is_virtual=False, updated_at=2023-10-16 15:04:08 UTC, storage_id=2, transform_id=1, run_id=1, created_by_id=1)

In [4]:
file.save()

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


In [5]:
file.storage

Storage(uid='oIYGbD74', root='s3://cellxgene-data-public', type='s3', region='us-west-2', updated_at=2023-11-20 12:37:11 UTC, created_by_id=1)

In [6]:
file.key

'cell-census/2023-07-25/h5ads/0d2ee4ac-05ee-40b2-afb6-ebb584caa867.h5ad'

In [7]:
file.path.download_to(f"./mytest/{file.key}")

In [8]:
!ls -alh ./mytest2/cell-census/2023-07-25/h5ads/

total 289M
drwxrwxr-x 2 ml4ig1 ml4ig1 4,0K nov.  21 22:38 .
drwxrwxr-x 3 ml4ig1 ml4ig1 4,0K nov.  21 22:37 ..
-rw-rw-r-- 1 ml4ig1 ml4ig1  16M nov.  21 22:38 1075c26e-5c47-43ba-902c-5b6cb864dd10.h5ad
-rw-rw-r-- 1 ml4ig1 ml4ig1  34M nov.  21 22:38 1492eb6b-7d50-4c4d-94ac-c801a7d5555c.h5ad
-rw-rw-r-- 1 ml4ig1 ml4ig1  37M nov.  21 22:38 b9b4cf27-9c22-410d-8bd8-5d43e379485b.h5ad
-rw-rw-r-- 1 ml4ig1 ml4ig1  22M nov.  21 22:38 cd3c693e-64e1-4009-a941-ba3f340c7434.h5ad
-rw-rw-r-- 1 ml4ig1 ml4ig1 182M nov.  21 22:38 d5c67a4e-a8d9-456d-a273-fa01adb1b308.h5ad


In [9]:
ln.Storage.filter().df()

,uid,root,type,region,updated_at,created_by_id
id,,,,,,
1,saIQmZUq,/home/ml4ig1/Documents code/scPRINT/notebooks/...,local,None,2023-11-20 12:36:53.843792+00:00,1
2,oIYGbD74,s3://cellxgene-data-public,s3,us-west-2,2023-11-20 12:37:11.585316+00:00,1


In [7]:
default_storage = ln.Storage.filter(root=ln.settings.storage.as_posix()).one()

In [5]:
default_storage

Storage(uid='d9usNZRL', root='/home/ml4ig1/Documents code/scPRINT/notebooks/assessments/mytest2', type='local', updated_at=2023-11-21 21:35:51 UTC, created_by_id=1)

In [12]:
file.storage = default_storage

In [13]:
file.save()

# from dataset

In [3]:
cx_dataset = ln.Collection.using("laminlabs/cellxgene-census").one()
cx_dataset

Dataset(uid='EAUF1AaT4kOVyHYnZsUJ', name='cellxgene-census', version='2023-07-25', hash='pEJ9uvIeTLvHkZW2TBT5', visibility=0, updated_at=2023-10-24 16:00:07 UTC, transform_id=1, run_id=9, created_by_id=2)

In [18]:
from django.db import IntegrityError

In [2]:
ln.Storage.filter().df()

,uid,root,type,region,updated_at,created_by_id
id,,,,,,
1,d9usNZRL,/home/ml4ig1/Documents code/scPRINT/notebooks/...,local,None,2023-11-21 21:35:51.871892+00:00,1
2,oIYGbD74,s3://cellxgene-data-public,s3,us-west-2,2023-11-21 21:37:12.693011+00:00,1


In [6]:
cx_dataset.files.all()[3].storage

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


Storage(uid='oIYGbD74', root='s3://cellxgene-data-public', type='s3', region='us-west-2', updated_at=2023-10-16 15:04:08 UTC, created_by_id=1)

In [9]:
files=[]
for file in cx_dataset.files.all()[:5]:
    file.save()
    #  print(f"File {file.uid} already exists in storage")
    # if location already has a file, don't save again
    #file.path.download_to(f"./mytest2/{file.key}")
    file.storage = default_storage
    file.save()
    files.append(file)

❗ no run & transform get linked, consider passing a `run` or calling ln.track()
❗ record with CAQPyYbcP4UjgiIfniQK already exists on default database: File(uid='CAQPyYbcP4UjgiIfniQK', key='cell-census/2023-07-25/h5ads/d5c67a4e-a8d9-456d-a273-fa01adb1b308.h5ad', suffix='.h5ad', accessor='AnnData', description='Retina|d5c67a4e-a8d9-456d-a273-fa01adb1b308', size=190765719, hash='d8ZhFETfIiYt51mH7_7CtQ-23', hash_type='md5-n', visibility=0, key_is_virtual=False, updated_at=2023-10-16 15:04:08 UTC, storage_id=2, transform_id=1, run_id=1, created_by_id=1)
❗ record with 3skUbr3O9875ELqMel6p already exists on default database: File(uid='3skUbr3O9875ELqMel6p', key='cell-census/2023-07-25/h5ads/1075c26e-5c47-43ba-902c-5b6cb864dd10.h5ad', suffix='.h5ad', accessor='AnnData', size=16579508, hash='OH3PUMiHF4zijtpdsBkbdw-2', hash_type='md5-n', visibility=0, key_is_virtual=False, updated_at=2023-10-16 15:04:08 UTC, storage_id=2, transform_id=1, run_id=1, created_by_id=1)
❗ record with U08Kty7hcUhsXiPc8

In [10]:
files[3].describe()

File(uid='1JZgXoQxMHfTMUax8Hho', key='cell-census/2023-07-25/h5ads/cd3c693e-64e1-4009-a941-ba3f340c7434.h5ad', suffix='.h5ad', accessor='AnnData', size=22613613, hash='lPzruZGc2nM2koFPosg3Ew-3', hash_type='md5-n', visibility=0, key_is_virtual=False, updated_at=2023-11-22 08:27:40 UTC)

Provenance:
  🗃️ storage: Storage(uid='d9usNZRL', root='/home/ml4ig1/Documents code/scPRINT/notebooks/assessments/mytest2', type='local', updated_at=2023-11-21 21:35:51 UTC, created_by_id=1)
  👤 created_by: User(uid='a3keNAVX', handle='jkobject', updated_at=2023-11-21 21:35:51 UTC)



In [12]:
dataset = ln.Collection(
    files, name="test_cxgene", description="first 10 cxgene files"
)
dataset.save()

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


In [13]:
dataset

Dataset(uid='Gc2K1R06aI7zjPqPZgAf', name='test_cxgene', description='first 10 cxgene files', hash='t5-vWhEkhRLTw2VgLX2Y', visibility=0, updated_at=2023-11-22 08:28:02 UTC, created_by_id=1)